<a href="https://colab.research.google.com/github/bobburudinesh/TinyML_Embedded_Systems/blob/main/Start_with_a_simple_neural_network_for_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start with a simple neural network for MNIST
Note that there are 2 layers, one with 20 neurons, and one with 10.

The 10-neuron layer is our final layer because we have 10 classes we want to classify.

Train this, and you should see it get about 98% accuracy

In [ ]:
# Load libraries
import sys

import tensorflow as tf

In [ ]:
# This script requires TensorFlow 2 and Python 3.
if sys.version_info.major < 3:
    raise Exception((f"The script is developed and tested for Python 3. "
                     f"Current version: {sys.version_info.major}"))

if tf.__version__.split('.')[0] != '2':
    raise Exception((f"The script is developed and tested for tensorflow 2. "
                     f"Current version: {tf.__version__}"))

In [ ]:
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

training_images  = training_images / 255.0
val_images = val_images / 255.0
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(20, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=20, validation_data=(val_images, val_labels))


## Examine the test data

Using model.evaluate, you can get metrics for a test set. In this case we only have a training set and a validation set, so we can try it out with the validation set. The accuracy will be slightly lower, at maybe 96%.  This is because the model hasn't previously seen this data and may not be fully generalized for all data. Still it's a pretty good score.

You can also predict images, and compare against their actual label. The [0] image in the set is a number 7, and here you can see that neuron 7 has a 9.9e-1 (99%+) probability, so it got it right!


In [ ]:

model.evaluate(val_images, val_labels)

classifications = model.predict(val_images)
print(classifications[0])
print(val_labels[0])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9548 - loss: 0.1664
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[8.3517515e-10 1.4485449e-11 2.8084357e-06 8.6936343e-05 3.8750068e-15
 7.7371672e-11 5.3942748e-16 9.9991012e-01 9.7888613e-09 1.2411363e-07]
7


## Modify to inspect learned values

This code is identical, except that the layers are named prior to adding to the sequential. This allows us to inspect their learned parameters later.

In [ ]:
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

training_images  = training_images / 255.0
val_images = val_images / 255.0
layer_1 = tf.keras.layers.Dense(20, activation=tf.nn.relu)
layer_2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    layer_1,
                                    layer_2])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=20)

model.evaluate(val_images, val_labels)

classifications = model.predict(val_images)
print(classifications[0])
print(val_labels[0])

# Inspect weights

If you print layer_1.get_weights(), you'll see a lot of data. Let's unpack it. First, there are 2 arrays in the result, so let's look at the first one. In particular let's look at its size.

In [ ]:
print(layer_1.get_weights()[0].size)

15680


The above code should print 15680. Why?

Recall that there are 20 neurons in the first layer.

Recall also that the images are 28x28, which is 784.

If you multiply 784 x 20 you get 15680.

So...this layer has 20 neurons, and each neuron learns a W parameter for each pixel. So instead of y=Mx+c, we have
y=M1X1+M2X2+M3X3+....+M784X784+C in every neuron!

Every pixel has a weight in every neuron. Those weights are multiplied by the pixel value, summed up, and given a bias.


In [ ]:
print(layer_1.get_weights()[1].size)

20


The above code will give you 20 -- the get_weights()[1] contains the biases for each of the 20 neurons in this layer.

## Inspecting layer 2

Now let's look at layer 2. Printing the get_weights will give us 2 lists, the first a list of weights for the 10 neurons, and the second a list of biases for the 10 neurons

Let's look first at the weights:

In [ ]:
print(layer_2.get_weights()[0].size)

200


This should return 200. Again, consider why?

There are 10 neurons in this layer, but there are 20 neurons in the previous layer. So, each neuron in this layer will learn a weight for the incoming value from the previous layer. So, for example, the if the first neuron in this layer is N21, and the neurons output from the previous layers are N11-N120, then this neuron will have 20 weights (W1-W20) and it will calculate its output to be:

W1N11+W2N12+W3N13+...+W20N120+Bias

So each of these weights will be learned as will the bias, for every neuron.

Note that N11 refers to Layer 1 Neuron 1.


In [ ]:
print(layer_2.get_weights()[1].size)

10


...and as expected there are 10 elements in this array, representing the 10 biases for the 10 neurons.

Hopefully this helps you see how the element of a simple neuron containing y=mx+c can be expanded greatly into a deep neural network, and that DNN can learn the parameters that match the 784 pixels of an image to their output!